In [0]:
import tensorflow as tf
import numpy as np
import sklearn
import pandas
import matplotlib.pyplot as plt

In [0]:
import numpy as np
from scipy import misc
import os
import matplotlib.pyplot as plt
import csv
def read_csv_infos(file):

	idx = []
	labels = []
	sizesequences = []

	with open(file, 'r') as f:
		data = csv.reader(f, delimiter=' ')
		for d in data:
			j = d[0].split(',')
			idx.append(int(j[0]))
			labels.append(int(j[1]))
			sizesequences.append(int(j[2]))

	return idx, labels, sizesequences

In [0]:
def read_csv_skeleton(file, dim_joints):

	n = []
	n_samples = 0
	n_frames = 0
	with open(file, 'r') as f:
		data = csv.reader(f, delimiter=' ')
		for d in data:
			n_samples += 1
			n_frames=0
			for j in d[0].split(','):
				n.append(float(j))
				n_frames += 1
    # convert data in numpy array
	n = np.asarray(n)
	n = np.reshape(n, (n_samples, n_frames//(22*dim_joints), 22, dim_joints))
	return n

In [44]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
skeletons2=read_csv_skeleton("/content/gdrive/My Drive/TPSs/data_challenge/skeletons_image_train.csv",2)
idx, labels, sequences=read_csv_infos("/content/gdrive/My Drive/TPSs/data_challenge/infos_train.csv")

In [46]:
skeletons2.shape

(1960, 171, 22, 2)

In [0]:
skeletons3=read_csv_skeleton("/content/gdrive/My Drive/TPSs/data_challenge/skeletons_world_train.csv",3)

In [48]:
skeletons3.shape

(1960, 171, 22, 3)

In [57]:
model = tf.keras.Sequential()

# Must define the input shape in the first layer of the neural network
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding="same", 
          input_shape=(171,22,3), activation='relu'))

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation='relu'))

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(3,3)))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.Dense(29, activation='softmax'))

# Take a look at the model summary
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 171, 22, 32)       896       
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 171, 22, 64)       18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 85, 11, 64)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 85, 11, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 85, 11, 128)       73856     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 83, 9, 256)        295168    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 27, 3, 256)       

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [0]:
labels=np.array(labels)

In [0]:

x=skeletons3
n=x.shape[0]
y=labels
selection=np.arange(n)
np.random.shuffle(selection)
subselection=selection[:int(n*0.6)]
x_train=x[subselection]
x_test=x[[i for i in np.arange(n) if i not in subselection]]
y_train=y[subselection]
y_test=y[[i for i in np.arange(n) if i not in subselection]]

In [61]:
model.fit(x,
         y,
         epochs=40,
          batch_size=64#,
          #validation_data=(x_test, y_test)
         )

Train on 1960 samples
Epoch 1/40
1960/1960 [==============================] - 3s 1ms/sample - loss: 3.2519 - acc: 0.0648
Epoch 2/40
1960/1960 [==============================] - 2s 1ms/sample - loss: 2.9760 - acc: 0.1082
Epoch 3/40
1960/1960 [==============================] - 3s 1ms/sample - loss: 2.7255 - acc: 0.1531
Epoch 4/40
1960/1960 [==============================] - 3s 1ms/sample - loss: 2.3844 - acc: 0.2291
Epoch 5/40
1960/1960 [==============================] - 3s 1ms/sample - loss: 2.1029 - acc: 0.3148
Epoch 6/40
1960/1960 [==============================] - 3s 1ms/sample - loss: 1.7963 - acc: 0.4061
Epoch 7/40
1960/1960 [==============================] - 3s 1ms/sample - loss: 1.5963 - acc: 0.4684
Epoch 8/40
1960/1960 [==============================] - 3s 1ms/sample - loss: 1.5057 - acc: 0.4990
Epoch 9/40
1960/1960 [==============================] - 3s 1ms/sample - loss: 1.3208 - acc: 0.5577
Epoch 10/40
1960/1960 [==============================] - 2s 1ms/sample - loss: 1.1795 -

In [54]:
model.evaluate(x_test,y_test)

784/784 [==============================] - 0s 562us/sample - loss: 1.4587 - acc: 0.7385


[1.4586967762027467, 0.7385204]

In [0]:
evaluation_data = read_csv_skeleton("/content/gdrive/My Drive/TPSs/data_challenge/skeletons_world_test.csv",3)

In [69]:
print(evaluation_data.shape)
print(skeletons3.shape)

(840, 162, 22, 3)
(1960, 171, 22, 3)


In [73]:
np.pad(evaluation_data, pad_width=9, constant_values=0, axis=1)

ValueError: ignored

In [0]:
model.predict(evaluation_data)